
This notebook provides examples of code to use word vectors to create a natural language type search facility in BigQuery. It accompanies a blog post which explains how it all works. 

You can find the blog post here:
https://www.efficientdatagroup.com/blog/natural-language-search-in-google-bigquery


In [2]:
from google.colab import auth
auth.authenticate_user()

Set the name of the project in the following cell. This will be inserted into SQL queries througohut the notebook. 

In [9]:
project_id = 'news-209921'
project_id = 'enter-your-project-name-here'

In [5]:
# Connect to the project and list datasets. This should contain a dataset called 'word_vectors_us'.

from google.cloud import bigquery
client = bigquery.Client(project=project_id)

for dataset in client.list_datasets():
  print(dataset.dataset_id)

word_vectors_us


## Run a word similarity query

In [13]:
word_sim = f"""WITH 
  glove AS (SELECT * FROM `{project_id}.word_vectors_us.glove_vectors`),

  search_vector AS (SELECT * FROM  glove 
            WHERE word = @search_word),

  cosine_sim AS (  
      SELECT  
          all_vectors.word,
          (SELECT #  u⋅v / (∥u∥ * ∥v∥)
              SUM(av_component * sv_component) / 
             ( SQRT(SUM(av_component * av_component)) * 
                  SQRT(SUM(sv_component * sv_component)) ) 
            FROM 
              UNNEST(all_vectors.vector)   av_component WITH OFFSET av_component_index
            JOIN
              UNNEST(search_vector.vector) sv_component WITH OFFSET sv_component_index
            ON av_component_index = sv_component_index
          ) AS cosine_similarity
    FROM glove AS all_vectors 
    CROSS JOIN search_vector
  )

SELECT * FROM cosine_sim
ORDER BY cosine_similarity DESC
LIMIT 10"""

In [14]:
# Choose the word you want to do the similarity search for below
search_term = "aubergine"
job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ScalarQueryParameter("search_word", "STRING", search_term),
    ]
)

In [15]:
job = client.query(word_sim, job_config=job_config)
res = job.result().to_dataframe()
res

,word,cosine_similarity
0,aubergine,1.000000
1,eggplant,0.814989
2,courgette,0.740594
3,aubergines,0.709632
4,beetroot,0.680378
5,caponata,0.653725
6,eggplants,0.646994
7,courgettes,0.644074
8,tapenade,0.621136
9,zucchini,0.618107


## Create tokeniser

In [16]:
tokeniser = f"""CREATE OR REPLACE FUNCTION `{project_id}.word_vectors_us.tokenise_no_stop`(text STRING) AS (
(
    SELECT ARRAY_AGG(word) FROM UNNEST(REGEXP_EXTRACT_ALL(
                                REGEXP_REPLACE(text, r''|\'s(\W)', r'\1'),
                                r'((?:\d+(?:,\d+)*(?:\.\d+)?)+|(?:[\w])+)')) AS word
    WHERE LOWER(word) not in UNNEST(['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', 'arent', 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', 'couldnt', 'd', 'did', 'didn', 'didnt', 'do', 'does', 'doesn', 'doesnt', 'doing', 'don', 'dont', 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', 'hadnt', 'has', 'hasn', 'hasnt', 'have', 'haven', 'havent', 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is', 'isn', 'isnt', 'it', 'its', 'itself', 'just', 'll', 'm', 'ma', 'me', 'mightn', 'mightnt', 'more', 'most', 'mustn', 'mustnt', 'my', 'myself', 'needn', 'neednt', 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', 'shant', 'she', 'shes', 'should', 'shouldn', 'shouldnt', 'shouldve', 'so', 'some', 'such', 't', 'than', 'that', 'thatll', 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 've', 'very', 'was', 'wasn', 'wasnt', 'we', 'were', 'weren', 'werent', 'what', 'when', 'where', 'which', 'while', 'who', 'whom', 'why', 'will', 'with', 'won', 'wont', 'wouldn', 'wouldnt', 'y', 'you', 'youd', 'youll', 'your', 'youre', 'yours', 'yourself', 'yourselves', 'youve'])
    )
);"""

In [ ]:
job = client.query(tokeniser)
res = job.result()

## Calculate word frequencies and save

In [17]:
word_frequencies = f"""CREATE OR REPLACE TABLE 
    `{project_id}.word_vectors_us.word_frequencies` AS

WITH tokens AS (
  SELECT `{project_id}.word_vectors_us.tokenise_text`(body) AS tk 
  FROM `bigquery-public-data.breathe.nature`
)
SELECT w AS word, COUNT(*) AS frequency
FROM tokens, UNNEST(tk) AS w
GROUP BY 1"""

In [ ]:
job = client.query(word_frequencies)
res = job.result()

## Create the document vectors

The following query creates document vectors for all the Nature articles in the breathe dataset. It is based on the body text.
Note that some of the article entries are missing body text, and so nothing is produced.

In [18]:
create_doc_vectors = f"""CREATE OR REPLACE TABLE `{project_id}.word_vectors_us.nature_vectors`
AS

WITH all_articles AS (SELECT  id,
                              date,
                              title, 
                              `{project_id}.word_vectors_us.tokenise_no_stop`(body) article 

FROM `bigquery-public-data.breathe.nature`),

# Unnest so we have one word per row
my_word_lists AS (
            SELECT  id, 
                    word 
            FROM all_articles, UNNEST(article) AS word 
   ),

# Fetch the vectors for each word and weight by word frequency
lookup_vectors AS (
  SELECT  
        id,
        word, 
        wv_component / POW(frequency, 0.4) AS tf_normalised_wv_component, 
        wv_component_index  
  FROM my_word_lists 
  JOIN `{project_id}.word_vectors_us.glove_vectors` 
    USING (word)
  JOIN `{project_id}.word_vectors_us.word_frequencies`
    USING (word)
  , UNNEST(vector) wv_component WITH OFFSET wv_component_index),

# Aggregate to create a document vector
aggregate_vectors_over_article AS (
  SELECT   id,
           wv_component_index, 
           SUM(tf_normalised_wv_component) aggregated_wv_component 
  FROM lookup_vectors 
  GROUP BY id, wv_component_index),

# Normalise vectors to unit length, so need to work out the length (magnitude)
vector_lengths AS (
  SELECT id, 
         SQRT(SUM(aggregated_wv_component * aggregated_wv_component)) AS magnitude 
  FROM aggregate_vectors_over_article
  GROUP BY id),

# Divide by the magnitude to normalise and put it back in to an array
normalised_aggregate_vectors AS (
    SELECT id,  
           ARRAY_AGG(aggregated_wv_component / magnitude ORDER BY wv_component_index) AS article_vector
    FROM aggregate_vectors_over_article 
    JOIN vector_lengths
      USING (id)
    GROUP BY id)

SELECT  id, 
        date, 
        title, 
        article_vector 
FROM normalised_aggregate_vectors
JOIN all_articles
USING (id)"""

In [ ]:
job = client.query(create_doc_vectors)
res = job.result()

## Create function to perform natural language search

In [19]:
my_search = f"""CREATE OR REPLACE PROCEDURE `{project_id}.word_vectors_us.similarity_search`(search_phrase STRING)
BEGIN

WITH 
  all_tokens AS (SELECT  `{project_id}.word_vectors_us.tokenise_no_stop`(search_phrase) tokens),

  # Unnest so we have one word per row
  tokens_no_stops AS (
            SELECT  word 
            FROM all_tokens, UNNEST(tokens) AS word 
   ),

  # Fetch vectors for each word, and weight by the word frequency in the corpus 
  lookup_vectors AS (SELECT  
                            word, 
                            vector_component / POW(frequency, 0.4) AS weighted_vector_component, 
                            vector_component_index  
  FROM tokens_no_stops 
  JOIN `{project_id}.word_vectors_us.glove_vectors` 
  USING (word)
  JOIN `{project_id}.word_vectors_us.word_frequencies`
  USING (word)
  , 
  UNNEST(vector) vector_component WITH OFFSET vector_component_index),

  # Aggregate the weighted components to generate a single vectors. Just take the SUM
  aggregate_vector AS (SELECT vector_component_index, SUM(weighted_vector_component) AS agg_vector_component 
                    FROM lookup_vectors 
                    GROUP BY vector_component_index),

  # We want to normalise the vectors to unit length, so need to work out the length (magnitude) of the 
  vector_length AS (SELECT SQRT(SUM(agg_vector_component * agg_vector_component)) AS magnitude 
  FROM aggregate_vector),

  # Divide by the magnitude to normalise and put it back in to an array
  norm_vector AS (
    SELECT ARRAY_AGG(agg_vector_component / (SELECT magnitude FROM vector_length) ORDER BY vector_component_index) AS vector
    FROM aggregate_vector)

SELECT  id,
        date,
        title,
        (SELECT SUM(doc_vector_component * search_vector_component) 
         FROM 
            UNNEST(dv.article_vector) doc_vector_component WITH OFFSET dv_component_index
         JOIN
            UNNEST(s.vector) search_vector_component WITH OFFSET sv_component_index
         ON dv_component_index = sv_component_index) AS cosine_similarity
FROM `{project_id}.word_vectors_us.nature_vectors` dv
CROSS JOIN norm_vector s
ORDER BY cosine_similarity DESC
LIMIT 10;
END;"""

In [ ]:
job = client.query(my_search)
res = job.result()

## Example search

In [20]:
job = client.query(f"""CALL `{project_id}.word_vectors_us.similarity_search`(
  "Epigenetics and cerebral organoids: promising directions in autism spectrum disorders"
);""" )

In [21]:
res = job.result().to_dataframe()

In [22]:
res

,id,date,title,cosine_similarity
0,5de2010b-bee3-4669-aa24-8287a8ab6fd9,2018-01-10,Epigenetics and cerebral organoids: promising ...,0.601886
1,361dd572-e745-4691-8822-232b052cb842,2018-05-30,Investigating pediatric disorders with induced...,0.597278
2,d779f221-da50-448f-9094-7450b0c67df8,2018-02-01,Modeling cancer using patient-derived induced ...,0.576579
3,230182c6-4362-4291-bb47-31a3e7b35d9c,2020-04-22,Epigenetic mechanism of SETDB1 in brain: impli...,0.571364
4,be8d2d29-b47e-4273-9867-7307853e02bc,2017-09-13,Toxoplasma Modulates Signature Pathways of Hum...,0.570963
5,ad05a0b9-d8c0-4042-a0bf-0bf79ce1b366,2018-06-11,Review and gap analysis: molecular pathways le...,0.564909
6,aa5ff4b3-86c5-4e72-b4c1-f121ea2a0ab9,2019-01-18,"Insights into genetics, human biology and dise...",0.563441
7,6c1d8d18-7c92-4f5f-96a5-582bf67f5e0d,2019-01-18,"Insights into genetics, human biology and dise...",0.563441
8,2f256d20-f728-4a06-81b7-716ed4534139,2018-02-26,Synaptic and transcriptionally downregulated g...,0.559447
9,8c839857-09dd-43b6-be3c-106e02f1e388,2020-02-28,Beyond the looking glass: recent advances in u...,0.558771
